<a href="https://colab.research.google.com/github/cbadenes/curso-pln/blob/main/notebooks/03_embeddings_sherlock_holmes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Embeddings de Palabras a partir de libros de Sherlock Holmes


# 1) Carga de Datos

Carga libros publicados en Project Gutenberg y elimina el header/footer:

In [1]:
import urllib.request
from pathlib import Path

def load_gutenberg_text(url):
    print("Cargando texto desde:", url, "..")
    response = urllib.request.urlopen(url)
    raw = response.read().decode('utf-8')

    # Encontrar el inicio y fin del contenido real (eliminar header/footer de Gutenberg)
    start = raw.find("*** START OF THE PROJECT GUTENBERG")
    start = raw.find("\n", start) + 1
    end = raw.find("*** END OF THE PROJECT GUTENBERG")

    return raw[start:end]

# URLs de los libros en Project Gutenberg
urls = [
    "https://www.gutenberg.org/files/1661/1661-0.txt",    # The Adventures of Sherlock Holmes
    "https://www.gutenberg.org/files/108/108-0.txt",      # The Return of Sherlock Holmes
    "https://www.gutenberg.org/files/2097/2097-0.txt",    # The Hound of the Baskervilles
    "https://www.gutenberg.org/files/244/244-0.txt"       # A Study in Scarlet
]

# Carga y guarda el texto
text = " "
for url in urls:
  book_text = load_gutenberg_text(url)
  text += book_text


print("Texto cargado:", len(text), "caracteres")

Cargando texto desde: https://www.gutenberg.org/files/1661/1661-0.txt ..
Cargando texto desde: https://www.gutenberg.org/files/108/108-0.txt ..
Cargando texto desde: https://www.gutenberg.org/files/2097/2097-0.txt ..
Cargando texto desde: https://www.gutenberg.org/files/244/244-0.txt ..
Texto cargado: 1742150 caracteres


# 2) Preprocesamiento

Tokeniza y limpia el texto:
* Convierte a minúsculas
* Elimina tokens que no son palabras

In [2]:
import spacy
from spacy.lang.en import English

# Inicializar spaCy (solo tokenizador para velocidad)
nlp = English(disable=['tagger','parser','ner'])

def tokenize(text):
    doc = nlp(text)
    return [token.text.lower() for token in doc
            if token.text.strip() and not token.is_punct]

# Dividir en oraciones y tokenizar
corpus_sentences = []
for line in text.split('\n'):
    if line.strip():  # ignorar líneas vacías
        tokens = tokenize(line)
        if tokens:  # ignorar líneas sin tokens válidos
            corpus_sentences.append(tokens)

print("Total de oraciones procesadas:", len(corpus_sentences))

Total de oraciones procesadas: 27683


# 3) Modelo Word2Vec

---



In [3]:
#!pip install numpy==1.26.4 scipy==1.13.1 gensim==4.3.3 spacy==3.7.5
from gensim.models import Word2Vec

##3.1) Entrenamiento

In [4]:
w2v_model = Word2Vec(sentences=corpus_sentences,
                    vector_size=300,      # Dimensión del vector (300)
                    window=8,             # Ventana más amplia para capturar más contexto (8)
                    min_count=2,          # Filtrar palabras poco frecuentes (5)
                    workers=4,
                    sg=1,                 # Usar Skip-gram
                    epochs= 30,           # Más ciclos de entrenamiento
                    negative= 15,         # Tamaño de Muestra Negativa (Negative sampling)
                    alpha= 0.025,         # Learning rate inicial
                    min_alpha= 0.0001     # Learning rate final
                  )

Guardar modelos

In [5]:
w2v_model.save("sherlock_w2v.model")

## 3.2) Análisis de Similitudes

Cargar vectores

In [6]:
w2v_vectors = w2v_model.wv

print("\nEstadísticas del modelo:")
print("Dimensión de los vectores:", w2v_vectors.vector_size)
print("Número de palabras (Word2Vec):", len(w2v_vectors.index_to_key))


Estadísticas del modelo:
Dimensión de los vectores: 300
Número de palabras (Word2Vec): 8416


In [7]:
print("\nPalabras más similares a 'holmes' (Word2Vec):")
print(w2v_vectors.most_similar('holmes'))


Palabras más similares a 'holmes' (Word2Vec):
[('sherlock', 0.5520164370536804), ('demurely', 0.4811103045940399), ('cheerily', 0.4448547959327698), ('gleefully', 0.44213828444480896), ('shrug', 0.4374382197856903), ('approvingly', 0.43318963050842285), ('bungler', 0.4272502064704895), ('inclusive', 0.42610833048820496), ('curtly', 0.425311416387558), ('triumphantly', 0.4236288368701935)]


In [8]:
print("\nPalabras más similares a 'crime' (Word2Vec):")
print(w2v_vectors.most_similar('crime'))



Palabras más similares a 'crime' (Word2Vec):
[('committed', 0.5089191794395447), ('records', 0.4670312702655792), ('literature', 0.4545971751213074), ('deliberate', 0.45210957527160645), ('featureless', 0.44890543818473816), ('perpetrator', 0.4422905445098877), ('talent', 0.4386681616306305), ('detect', 0.43413999676704407), ('sots', 0.42703020572662354), ('insane', 0.42568156123161316)]


In [9]:
print("\nSimilitud entre 'crime' y 'art':")
print("Word2Vec:", w2v_vectors.similarity('crime', 'art'))


Similitud entre 'crime' y 'art':
Word2Vec: 0.2106922


## 3.3) Palabras fuera de vocabulario

In [10]:
print("\nPrueba con palabra fuera de vocabulario:")
try:
    print("Word2Vec - Similares a 'investigador':")
    print(w2v_vectors.most_similar('investigador'))
except KeyError:
    print("Word2Vec no puede manejar palabras fuera de vocabulario")


Prueba con palabra fuera de vocabulario:
Word2Vec - Similares a 'investigador':
Word2Vec no puede manejar palabras fuera de vocabulario


## 3.4) Analogías

In [11]:
print("\nAnalogías (Word2Vec):")
result = w2v_vectors.most_similar(positive=['watson', 'crime'],
                                negative=['holmes'])
print("watson:holmes como crime:?")
print(result)


Analogías (Word2Vec):
watson:holmes como crime:?
[('contemplation', 0.305237352848053), ('capable', 0.3048299551010132), ('waylaid', 0.29584941267967224), ('comparison', 0.29572513699531555), ('trials', 0.2947629392147064), ('player', 0.292878657579422), ('featureless', 0.29119187593460083), ('sots', 0.2911495268344879), ('unique', 0.29109957814216614), ('reliable', 0.2862240672111511)]


#4) Modelo FastText


In [12]:
from gensim.models import FastText

##4.1) Entrenamiento

In [13]:
ft_model = FastText(sentences=corpus_sentences,
                   vector_size=300,    # Aumentar dimensionalidad (300)
                   window=8,           # Ventana más amplia para capturar más contexto (8)
                   min_count=2,        # Mantener min_count bajo para capturar más variantes
                   workers=4,
                   sg=1,               # Skip-gram para mejor calidad
                   min_n=2,            # Tamaño mínimo de n-gramas
                   max_n=6,            # Tamaño máximo de n-gramas (aumentado para capturar más patrones)
                   epochs=30,          # Más ciclos de entrenamiento (30)
                   word_ngrams=1,      # Habilitar n-gramas de palabras
                   negative=15,        # Más muestras negativas
                   alpha=0.025,        # Learning rate inicial
                   min_alpha=0.0001    # Learning rate final
              )

Almacenamiento del modelo:

In [14]:
ft_model.save("sherlock_ft.model")

## 4.2) Análisis de Similitudes

In [15]:
ft_vectors = ft_model.wv

print("\nEstadísticas del modelo:")
print("Dimensión de los vectores:", ft_vectors.vector_size)
print("Número de palabras (FastText):", len(ft_vectors.index_to_key))


Estadísticas del modelo:
Dimensión de los vectores: 300
Número de palabras (FastText): 8416


In [16]:
print("\nPalabras más similares a 'holmes' (FastText):")
print(ft_vectors.most_similar('holmes'))


Palabras más similares a 'holmes' (FastText):
[('holes', 0.4611356258392334), ('sherlock', 0.4445723295211792), ('soames', 0.43678635358810425), ('holborn', 0.4315681457519531), ('volumes', 0.4102344810962677), ('hold', 0.4083307981491089), ('holy', 0.4080694019794464), ('apologies', 0.3960151672363281), ('holds', 0.394469290971756), ('fumes', 0.3857247531414032)]


In [17]:
print("\nPalabras más similares a 'crime' (Word2Vec):")
print(ft_vectors.most_similar('crime'))


Palabras más similares a 'crime' (Word2Vec):
[('crimes', 0.804222822189331), ('criticism', 0.653266191482544), ('grime', 0.6352939009666443), ('crib', 0.6334989070892334), ('criminal', 0.6279764771461487), ('criminals', 0.6214247345924377), ('prime', 0.6146371960639954), ('crimson', 0.6062003374099731), ('cripple', 0.5827522873878479), ('crisis', 0.5692859888076782)]


In [18]:
print("\nSimilitud entre 'crime' y 'art':")
print("FastText:", ft_vectors.similarity('crime', 'art'))


Similitud entre 'crime' y 'art':
FastText: 0.0763179


##4.3) Palabras fuera del vocabulario

In [19]:
print("\nPrueba con palabra fuera de vocabulario:")

print("\nFastText - Similares a 'investigador':")
print(ft_vectors.most_similar('investigador'))  # FastText puede generar vectores para palabras nuevas



Prueba con palabra fuera de vocabulario:

FastText - Similares a 'investigador':
[('investigate', 0.9274206757545471), ('investigated', 0.9121105074882507), ('investigating', 0.8989437818527222), ('investigations', 0.8705874085426331), ('invest', 0.8689622282981873), ('investigation', 0.8682444095611572), ('investments', 0.7567822933197021), ('domestic', 0.5960216522216797), ('testimonial', 0.5802162885665894), ('destiny', 0.5447513461112976)]
